# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [ ]:
from azureml.core import Workspace, Experiment
from azureml.widgets import RunDetails
from azureml.train.automl import AutoMLConfig
import joblib, pickle

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [ ]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'AutoMLExperimentCapstone'

experiment=Experiment(ws, experiment_name)

ds = pd.read_csv('mileage.csv')

from train_FINAL import clean_data

# Use the clean_data function to cln your data.
x, y = clean_data(ds)
cleands = x.join(y)

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [ ]:
# TODO: Put your automl settings here
automl_settings = {experiment_timeout_minutes=25,
    task='regression',
    primary_metric='accuracy',
    training_data=cleands,
    label_column_name='mpg',
    n_cross_validations=3}

# TODO: Put your automl config here
automl_config = AutoMLConfig(automl_settings)

In [ ]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)
remote_run.wait_for_completion(show_output=True)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
RunDetails(remote_run).show()

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [ ]:
best,fitted = remote_run.get_output()
print(best)

In [ ]:
print(fitted)

In [ ]:
#TODO: Save the best model
filename = 'finalized_model.sav'
pickle.dump(best, open(filename, 'wb'))

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
# Register the model
registered_model = remote_run.register_model(model_name='finalized_model.sav',
                           tags={'area': 'mileage'},
                           model_path='best_automl.pkl')
print(registered_model.name, registered_model.id, registered_model.version, sep='\t')

#Create inference config
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig

env = Environment.get(ws, "RangaENV")

inference_config = InferenceConfig(entry_script='train_FINAL.py',
                                    environment=env)

#Deploy model and check service state
from azureml.core.webservice import LocalWebservice, Webservice

deployment_config = LocalWebservice.deploy_configuration(port=9030) #No authentication.
service = Model.deploy(ws, "deployed", [registered_model], inference_config, deployment_config)
service.wait_for_deployment(show_output = True)
print(service.state)
print(service.scoring_uri)
print(service.swagger_uri)

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
import requests
import json

# URL for the web service
scoring_uri = '<your web service URI>'

# Two sets of data to score, so we get two results back
data = {"data":
            [
                18,
                4,
                122,
                87,
                2332,
                12,
                70,
                2,
                'chevrolet chevelle malibu'
            ]
        
        }
# Convert to JSON string
input_data = json.dumps(data)

# Set the content type
headers = {'Content-Type': 'application/json'}
# If authentication is enabled, set the authorization header
headers['Authorization'] = f'Bearer {key}'

# Make the request and display the response
resp = requests.post(scoring_uri, input_data, headers=headers)
print(resp.text)

TODO: In the cell below, print the logs of the web service and delete the service